In [1]:
!pip install pandas numpy matplotlib seaborn


In [2]:
import pandas as pd
import numpy as np
# Load datasets
diabetes = pd.read_excel(r"C:\Users\Lenovo\Downloads\diabetes.xlsx")
covid = pd.read_excel(r"C:\Users\Lenovo\Downloads\covid.xlsx")
# View first rows
print(diabetes.head())
print(covid.head())



   Pregnancies  Glucose  Blood pressure  Skin thickness  Insulin  \
0            6      148              72              35        0   
1            1       85              66              29        0   
2            8      183              64               0        0   
3            1       89              66              23       94   
4            0      137              40              35      168   

   Body mass index  Diabetes pedigree function  Age  Outcome  
0             33.6                       0.627   50        1  
1             26.6                       0.351   31        0  
2             23.3                       0.672   32        1  
3             28.1                       0.167   21        0  
4             43.1                       2.288   33        1  
  Date_reported Country_code      Country WHO_region  New_cases  \
0    2020-01-03           AF  Afghanistan       EMRO          0   
1    2020-01-04           AF  Afghanistan       EMRO          0   
2    2020-01

In [3]:
import numpy as np
# Step 1: Check column names
print(diabetes.columns)
# Step 2: Replace invalid zeros
cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in cols:
    if col in diabetes.columns:
        diabetes[col] = diabetes[col].replace(0, np.nan)
# Step 3: Fill missing numeric values only
diabetes.fillna(diabetes.median(numeric_only=True), inplace=True)
# Step 4: Create age groups
diabetes['Age_Group'] = pd.cut(
    diabetes['Age'],
    bins=[20, 30, 40, 50, 60, 80],
    labels=['20-30', '30-40', '40-50', '50-60', '60+']
)
# Step 5: Add simulated year
diabetes['Year'] = np.random.choice(
    [2017, 2018, 2019, 2020, 2021, 2022, 2023],
    size=len(diabetes)
)
# Preview cleaned data
print(diabetes.head())


Index(['Pregnancies', 'Glucose', 'Blood pressure', 'Skin thickness', 'Insulin',
       'Body mass index', 'Diabetes pedigree function', 'Age', 'Outcome'],
      dtype='object')
   Pregnancies  Glucose  Blood pressure  Skin thickness  Insulin  \
0            6    148.0              72              35    125.0   
1            1     85.0              66              29    125.0   
2            8    183.0              64               0    125.0   
3            1     89.0              66              23     94.0   
4            0    137.0              40              35    168.0   

   Body mass index  Diabetes pedigree function  Age  Outcome Age_Group  Year  
0             33.6                       0.627   50        1     40-50  2019  
1             26.6                       0.351   31        0     30-40  2022  
2             23.3                       0.672   32        1     30-40  2018  
3             28.1                       0.167   21        0     20-30  2021  
4             43.1 

In [4]:
def covid_phase(year):
    if year <= 2019:
        return "Pre-COVID"
    elif year <= 2021:
        return "During COVID"
    else:
        return "Post-COVID"
diabetes['Phase'] = diabetes['Year'].apply(covid_phase)


In [5]:
import pandas as pd

# Load Excel file
covid = pd.read_excel(r"C:\Users\Lenovo\Downloads\covid.xlsx")

# STEP 1: Clean column names (remove spaces & lowercase)
covid.columns = covid.columns.str.strip().str.lower()

# STEP 2: Print columns so we can see what exists
print("Available columns:", covid.columns.tolist())

# STEP 3: Try to auto-detect correct column names
date_col = [c for c in covid.columns if "date" in c][0]
cases_col = [c for c in covid.columns if "case" in c][0]
deaths_col = [c for c in covid.columns if "death" in c][0]

# STEP 4: Select columns safely
covid = covid[[date_col, cases_col, deaths_col]].copy()
covid.columns = ["date", "cases", "deaths"]

# STEP 5: Convert date
covid["date"] = pd.to_datetime(covid["date"], errors="coerce")

# STEP 6: Extract year
covid["Year"] = covid["date"].dt.year

# STEP 7: Group yearly
covid_yearly = covid.groupby("Year")[["cases", "deaths"]].sum().reset_index()

print(covid_yearly.head())


Available columns: ['date_reported', 'country_code', 'country', 'who_region', 'new_cases', 'cumulative_cases', 'new_deaths', 'cumulative_deaths']
   Year     cases   deaths
0  2020  82629525  1813198
1  2021  82528760  1611819


In [6]:

final_data = pd.merge(diabetes, covid_yearly,
                      on='Year', how='left')

In [7]:
final_data.to_csv(r"C:\Users\Lenovo\Downloads\cleaned_diabetes_covid.csv", index=False)
